# Image Processing: Histogram of Oriented Gradients (HOG) with MNIST Dataset

In [101]:
import torch
import torch.nn as nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from sklearn.preprocessing import OneHotEncoder
from torchvision import transforms
from skimage.transform import resize
from skimage.feature import hog
import numpy as np
import matplotlib.pyplot as plt

In [84]:
def transform(input):
    features = hog(
        input,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2,2),
    )

    # convert numpy to tensor
    return torch.from_numpy(features).type(torch.FloatTensor)

# get training and test set
mnist_train = MNIST("./data/", download = True, transform = transform)
mnist_test = MNIST("./data/", train = False, download = True, transform = transform)

In [86]:
batch_size = 128
train_loader = DataLoader(mnist_train, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(mnist_test, batch_size = batch_size, shuffle = True)

In [103]:
# classification with SVM loss
svm = nn.Linear(144, 10)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(svm.parameters(), lr = 0.01)

num_epochs = 2
for it in range(num_epochs):
    for imgs, labels in train_loader:
        # one hot encoder for loss
        enc = OneHotEncoder(
            handle_unknown = "ignore",
            sparse = False,
        )
        labels = enc.fit_transform(labels.detach().numpy().reshape(-1, 1))

        print(imgs, labels)
        print(imgs.shape, labels.shape)

        # forward pass
        output = svm(imgs)

        print(output.shape)
        loss = criterion(output, labels)

        # backward pass
        loss.backward()

        # update
        optimizer.step()
        optimizer.zero_grad()

tensor([[0.0248, 0.0000, 0.0932,  ..., 0.1217, 0.2500, 0.1444],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.1736, 0.1219, 0.0406],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0156, 0.0597, 0.0000]]) [[0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]]
torch.Size([128, 144]) (128, 10)
torch.Size([128, 10])


TypeError: multilabel_margin_loss(): argument 'target' (position 2) must be Tensor, not numpy.ndarray